# External Tools: Wrappers and Adapters

This tutorial defines the **supported integration pattern** for external tools.

MolPy uses two peer layers:

`molpy.wrapper`: run CLIs/binaries via `subprocess` and `molpy.adapter`: synchronize representations, no subprocess.

Workflow decisions, ordering, parameter selection, artifact paths belong in workflow/compute code — not inside wrappers/adapters.


## Decision table

Need to call a binary? → **Wrapper**, Need to bridge representations? → **Adapter**, and Need to decide steps/order/files? → **Workflow/compute**.


## Wrapper quickstart

Wrappers are safe to instantiate and inspect without executing optional tools.
Here we use `echo` so the docs remain runnable on any machine.


In [ ]:
from pathlib import Path

from molpy.wrapper.base import Wrapper

w = Wrapper(name="echo", exe="echo", workdir=Path("./_doc_tmp/echo"))
p = w.run(args=["hello", "wrapper"])
print((p.stdout or '').strip())


### Real wrapper example

This snippet shows the intended pattern for AmberTools wrappers. It is **not executed** in docs because AmberTools is an optional dependency:

```python
from pathlib import Path
from molpy.wrapper import AntechamberWrapper

w = AntechamberWrapper(
 name='antechamber',
 workdir=Path, 'work/antechamber',
 conda_env='AmberTools25',
)
w.check()
# proc = w.run_raw
```


## Adapter quickstart

Adapters are deterministic synchronization logic. They must not spawn subprocesses.
Below is a minimal adapter that round-trips a tiny key/value format.


In [ ]:
from molpy.adapter.base import Adapter


class KVAdapter(Adapter[dict, str]):
    def sync_to_internal(self) -> None:
        super().sync_to_internal()
        assert self._external is not None
        out = {}
        for item in [p for p in self._external.split(';') if p]:
            k, v = item.split('=')
            out[k] = v
        self._internal = out

    def sync_to_external(self) -> None:
        super().sync_to_external()
        assert self._internal is not None
        self._external = ';'.join(f"{k}={v}" for k, v in sorted(self._internal.items()))


a = KVAdapter(external='a=1;b=2')
print(a.get_internal())
a.set_internal({'z': '9', 'a': '1'})
print(a.get_external())


### Optional RDKit adapter

MolPy provides an optional `RDKitAdapter` bridge. Because RDKit is optional, keep it behind your own optional-install boundary:

```python
from molpy.adapter import RDKitAdapter

adapter = RDKitAdapter, internal=atomistic
mol = adapter.get_external()
# ... RDKit operations ...
adapter.set_external, mol
atomistic2 = adapter.get_internal()
```


## Putting it together

A typical integration flow is:

1) workflow code defines artifact paths
2) adapter/IO writes inputs, or bridges in-memory
3) wrapper runs the external tool
4) adapter/IO reads results back into MolPy

This keeps subprocess side effects isolated and keeps adapters deterministic.


## Compatibility: `molpy.external`

`molpy.external` is a temporary compatibility facade that re-exports the wrapper/adapter APIs.
Prefer direct imports from `molpy.wrapper` and `molpy.adapter` for new code.
